In [1]:
import random
import time
from graph.graph import read_graph_from_csv

In [2]:
from stable_matching.TaskWorker import outward_satisfactory,overall_satisfactory,gamma_workers,individual_rationality_tasks
def estimate(tasks, workers):
    result_dict = {'outward_satisfactory': outward_satisfactory(tasks, workers),
                   # 'overall_satisfactory': overall_satisfactory(tasks,workers),
                   'individual_rationality': individual_rationality_tasks(tasks)}
    Sum = 0
    Max = 0
    Min = 1000
    for t_ in tasks:
        q = t_.Q * gamma_workers(t_.R, t_.students()) / len(t_.R) / t_.budget 
        Sum += q
        if q> Max:
            Max = q
        if q< Min:
            Min = q
    result_dict['average_density'] = Sum/len(tasks)
    result_dict['maximum_density'] = Max
    result_dict['minimum_density'] = Min
    return result_dict

In [3]:
graph_file = '../graphs/dash/dash.csv'

In [4]:
m = 10 # number of tasks
n = 100 # number of candidate workers

initialize workers by their ids

In [5]:
from stable_matching.TaskWorker import Worker

g = read_graph_from_csv(graph_file,0)
workers = []
worker_ids = random.sample(list(g.nodes.keys()),n)
for i in range(n):
    workers.append(Worker(idx = worker_ids[i]))
del g

initialize tasks : (ids, budget, reverse reachable set, Q)

In [6]:
from graph.QIM import sampling
from utils.funcs import max_k
from stable_matching.TaskWorker import Task
import scipy.stats as stats

mu = 1.0
sigma = 0.1

tasks = []
graph_ids = random.sample(range(100),m)

# allocate budget to every task, sum of all budget is total_budget
avg_budget  = 10
total_budget = avg_budget * m
points = [0]+sorted(random.sample(range(1,total_budget),m-1))+[total_budget]
budgets = [points[i+1]-points[i] for i in range(len(points)-1)]

for i in range(m):
    G = read_graph_from_csv(graph_file,graph_ids[i])
    
    # generate cost of all candidate worker, task cost is generate from a trunc gaussian distribution mu = 1.0, sigma = 0.1, lb = 0.5, ub = 1.5
    X = stats.truncnorm((0.5-mu)/sigma, (1.5-mu)/sigma, loc= mu, scale= sigma).rvs(n)
    costs = {}
    for j in range(n):
        costs[workers[j]] = X[j]
        
    # values of all workers
    values = {}
    Q  = 0.0
    for v in G.nodes:
        values[v] = random.uniform(0.0, 1.0)
        Q += values[v]
        
    budget = budgets[i]
    print('budget:{}'.format(budget))
    start = time.time()
    # generate hyper graph of reverse reachable set in graph G
    k = max_k(budget, costs)
    print('k:{}'.format(k))
    RR = sampling(graph=G, C=worker_ids, k=k, delta=1/n, epsilon=0.01, values=values)
    print('time to generate RR:{}'.format(time.time()-start))
    
    #initialize tasks
    tasks.append(Task(idx=i, budget=budget, R=RR, Q=Q))
    tasks[i].initialize(costs)

budget:3
k:3
i_max:36
time to generate RR:2.0834457874298096
budget:10
k:12
i_max:35
time to generate RR:1.590407133102417
budget:11
k:13
i_max:34
time to generate RR:1.6999993324279785
budget:14
k:16
i_max:34
time to generate RR:0.8882865905761719
budget:7
k:8
i_max:35
time to generate RR:1.4123830795288086
budget:16
k:18
i_max:34
time to generate RR:0.9553320407867432
budget:7
k:8
i_max:35
time to generate RR:2.856632947921753
budget:4
k:4
i_max:36
time to generate RR:2.4106717109680176
budget:22
k:25
i_max:34
time to generate RR:0.5468084812164307
budget:6
k:7
i_max:35
time to generate RR:1.357759714126587


In [7]:
for worker in workers:
    value_dict = {}
    for t in tasks:
        value_dict[t] = random.random()
    worker.set_preference(value_dict)

In [20]:
from stable_matching.stableMatching import generalized_da
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
    task.set_choice_budget()
generalized_da(tasks,workers)

In [21]:
estimate(tasks, workers)

{'outward_satisfactory': 0.9795918367346939,
 'individual_rationality': [[3, 2.9718015257772255],
  [10, 9.994128139015812],
  [11, 10.847116544265768],
  [14, 12.927014162584289],
  [7, 6.84391689927044],
  [16, 15.354596589409484],
  [7, 6.211169100913683],
  [4, 3.2224570498522076],
  [22, 21.90560819594582],
  [6, 5.7591655267193484]],
 'average_density': 1.3769793216641504,
 'maximum_density': 2.610171302597001,
 'minimum_density': 0.6152048564323909}

In [15]:
from stable_matching.stableMatching import generalized_da
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
    task.set_choice_max_cover()
generalized_da(tasks,workers)

In [16]:
estimate(tasks,workers)

{'outward_satisfactory': 0.7,
 'individual_rationality': [[3, 0.8092816258322615],
  [10, 1.0549531202681048],
  [11, 0.7674477687465863],
  [14, 0.8644039330299084],
  [7, 0.7597340029346388],
  [16, 0.7746205534288589],
  [7, 0.9335383536255147],
  [4, 0.8905699832942072],
  [22, 0.9781958938013107],
  [6, 0.9725541151799945]],
 'average_density': 0.8149436898230892,
 'maximum_density': 2.721242421856448,
 'minimum_density': 0.2590336237610067}

In [19]:
from stable_matching.stableMatching import generalized_da
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
    task.set_choice_matroid(4)
generalized_da(tasks,workers)
estimate(tasks,workers)

{'outward_satisfactory': -0.18421052631578938,
 'individual_rationality': [[3, 0.930991066746693],
  [10, 4.819081824086796],
  [11, 4.620820181152417],
  [14, 4.899225645082625],
  [7, 2.778741421409652],
  [16, 4.615298206138719],
  [7, 3.531452191633326],
  [4, 1.9496563497753931],
  [22, 5.0913981820862055],
  [6, 2.7192511436413294]],
 'average_density': 1.3114202781124262,
 'maximum_density': 2.5771902540827107,
 'minimum_density': 0.77710087128302}

In [22]:
from stable_matching.stableMatching import heuristic
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
heuristic(tasks,workers,100)
estimate(tasks,workers)

ValueError: list.remove(x): x not in list